In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import length, rand, randn, mean, min, max, avg, stddev, sum
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [2]:
#Start spark session
spark = (SparkSession
        .builder
        .appName("TrabalhoPratico")
        .getOrCreate())

23/01/31 02:02:31 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/31 02:02:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 02:02:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("./all_stocks_5yr.csv"))
df.show(10)

+-------------------+-----+-----+-----+-----+--------+----+
|               date| open| high|  low|close|  volume|Name|
+-------------------+-----+-----+-----+-----+--------+----+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15 00:00:00|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19 00:00:00|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20 00:00:00|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21 00:00:00|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22 00:00:00|13.57| 13.6|13.21|13.57| 6071400| AAL|
+-------------------+-----+-----+-----+-----+--------+----+
only showing top 10 rows



In [4]:
# Quantos registros há na planilha?
df.count()

619040

In [5]:
# Quantos registros há na planilha para a ação da Apple (AAPL)?
apple = df.filter(df.Name == "AAPL")
apple.count()

1259

In [6]:
# Quantas empresas distintas têm registros nessa planilha?
distinctNames = df.groupBy("Name").agg(avg("open"))
distinctNames.count()
# distinctNames.show(20)

505

In [7]:
#Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?
(df.filter(df.close > df.open).count() / df.count()) * 100


51.52655724993538

In [8]:
#Qual o maior valor das ações da Apple (AAPL) na história?
df.filter(df.Name == "AAPL").select(max('high')).show()


+---------+
|max(high)|
+---------+
|    180.1|
+---------+



In [9]:
#Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.
df_stddev = df.groupBy("Name").agg(stddev("close").alias("stddev_close"))
df_stddev.sort(df_stddev.stddev_close.desc()).show()

+-----+------------------+
| Name|      stddev_close|
+-----+------------------+
| PCLN|  320.533473018748|
| AMZN|  282.500395033522|
|GOOGL|187.57389185479704|
| GOOG|166.42052902768182|
|  MTD|131.39721989584996|
|  AZO|130.74959678201296|
|  CMG|130.28062354899413|
| EQIX| 95.21280494966273|
| REGN| 94.06934511650653|
| CHTR| 83.63699908227738|
| ISRG| 72.97599882123069|
| AVGO| 71.41622240006728|
| ULTA| 67.15184610039526|
|  NOC| 66.39592749504422|
|  SHW| 64.52058470708296|
|  LMT| 62.41202936228181|
|  BLK| 62.33688426948667|
| NVDA| 59.62464184686514|
| ORLY| 59.09508035311545|
|  AGN|59.026349830314544|
+-----+------------------+
only showing top 20 rows



In [10]:
#Qual o dia com maior volume total de negociação da bolsa?
df_volumeByDay = df.groupBy("date").agg(sum("volume").alias("sum_volume"))
df_volumeByDay.sort(df_volumeByDay.sum_volume.desc()).show()

+-------------------+----------+
|               date|sum_volume|
+-------------------+----------+
|2015-08-24 00:00:00|4607945196|
|2016-06-24 00:00:00|4367393052|
|2015-12-18 00:00:00|4124454411|
|2016-01-20 00:00:00|4087629753|
|2018-02-06 00:00:00|4072080890|
|2016-11-10 00:00:00|4060601612|
|2014-10-15 00:00:00|3993171524|
|2013-06-21 00:00:00|3983923288|
|2015-09-18 00:00:00|3962050449|
|2016-11-09 00:00:00|3915089371|
|2016-01-15 00:00:00|3787884056|
|2017-12-15 00:00:00|3786992731|
|2016-02-08 00:00:00|3759709109|
|2016-03-18 00:00:00|3755746256|
|2016-01-29 00:00:00|3740170664|
|2014-03-21 00:00:00|3611991561|
|2018-02-05 00:00:00|3598437288|
|2013-03-15 00:00:00|3569550899|
|2016-12-16 00:00:00|3564871264|
|2016-02-11 00:00:00|3547898496|
+-------------------+----------+
only showing top 20 rows



In [11]:
#Qual a ação mais negociada da bolsa, em volume de transações?
df_volume = df.groupBy("Name").agg(sum("volume").alias("sum_volume"))
df_volume.sort(df_volume.sum_volume.desc()).show(1)

+----+------------+
|Name|  sum_volume|
+----+------------+
| BAC|117884953591|
+----+------------+
only showing top 1 row



In [12]:
#Quantas ações começam com a letra “A”?
distinctNames.filter(df.Name.startswith('A')).count()
#distinctNames.filter(df.Name.startswith('A')).show()

59

In [13]:
#Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?
(df.filter(df.high == df.close).count() / df.count()) * 100

1.1986301369863013

In [14]:
#Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?
diff_apple = apple.withColumn("diff", apple.close - apple.open)
diff_apple.sort(diff_apple.diff.desc()).show()

+-------------------+-------+-------+-------+-------+---------+----+------------------+
|               date|   open|   high|    low|  close|   volume|Name|              diff|
+-------------------+-------+-------+-------+-------+---------+----+------------------+
|2015-08-24 00:00:00|  94.87|  108.8|   92.0| 103.12|162206292|AAPL|              8.25|
|2018-02-06 00:00:00| 154.83| 163.72|  154.0| 163.03| 68243838|AAPL| 8.199999999999989|
|2017-05-08 00:00:00| 149.03|  153.7| 149.03| 153.01| 48752413|AAPL|3.9799999999999898|
|2014-12-09 00:00:00| 110.19|  114.3| 109.35| 114.12| 60208036|AAPL| 3.930000000000007|
|2017-10-27 00:00:00| 159.29|  163.6|  158.7| 163.05| 44454160|AAPL|3.7600000000000193|
|2015-12-04 00:00:00| 115.29| 119.25| 115.11| 119.03| 57776977|AAPL| 3.739999999999995|
|2017-12-11 00:00:00|  169.2| 172.89| 168.79| 172.67| 35273759|AAPL| 3.469999999999999|
|2016-11-16 00:00:00|  106.7| 110.23|  106.6| 109.99| 58840522|AAPL| 3.289999999999992|
|2015-07-22 00:00:00| 121.99|  1

In [15]:
#Em média, qual o volume diário de transações das ações da AAPL?
apple.agg(avg("volume")).show()

+-------------------+
|        avg(volume)|
+-------------------+
|5.404789973550437E7|
+-------------------+



In [16]:
#Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?
print(distinctNames.where(length("Name") == 1).count())
print(distinctNames.where(length("Name") == 2).count())
print(distinctNames.where(length("Name") == 3).count())
print(distinctNames.where(length("Name") == 4).count())
print(distinctNames.where(length("Name") == 5).count())

10
50
323
117
5


In [17]:
#Qual a ação menos negociada da bolsa, em volume de transações?
df_volumeByDay = df.groupBy("Name").agg(sum("volume").alias("sum_volume"))
df_volumeByDay.sort(df_volumeByDay.sum_volume.asc()).show(1)

+----+----------+
|Name|sum_volume|
+----+----------+
|APTV|  92947779|
+----+----------+
only showing top 1 row



In [18]:
spark.stop()